## Building basic likelihoods

In [3]:
import sys
import os
import glob

import tqdm

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

%matplotlib inline

In [5]:
import jax
jax.config.update('jax_enable_x64', True)

import jax.random
import jax.numpy as jnp

In [6]:
import discovery as ds
import discovery.models.nanograv as ds_nanograv
import discovery.samplers.numpyro as ds_numpyro

Read nanograv pulsars

In [8]:
allpsrs = [ds.Pulsar.read_feather(psrfile) for psrfile in sorted(glob.glob('../data/*-[JB]*.feather'))]

In [10]:
psr = allpsrs[0]

### Single pulsar likelihood

#### Measurement noise only, no backends

In [11]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement_simple(psr)])

What are the active parameters?

In [14]:
m.logL.params

['B1855+09_efac', 'B1855+09_log10_t2equad']

Sample random values from their priors

In [16]:
p0 = ds.sample_uniform(m.logL.params); p0

{'B1855+09_efac': 0.9631259853031077,
 'B1855+09_log10_t2equad': -8.161514352406641}

Evaluate the likelihood

In [17]:
m.logL(p0)

Array(89964.27597335, dtype=float64)

Try compiled version, grad

In [18]:
jax.jit(m.logL)(p0)

Array(89964.27597335, dtype=float64)

In [19]:
jax.grad(m.logL)(p0)

{'B1855+09_efac': Array(25210.26046491, dtype=float64, weak_type=True),
 'B1855+09_log10_t2equad': Array(250.88301607, dtype=float64, weak_type=True)}

#### Measurement noise only, nanograv backends, free parameters

In [22]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr)])

In [23]:
m.logL.params

['B1855+09_430_ASP_efac',
 'B1855+09_430_PUPPI_efac',
 'B1855+09_L-wide_ASP_efac',
 'B1855+09_L-wide_PUPPI_efac',
 'B1855+09_430_ASP_log10_t2equad',
 'B1855+09_430_PUPPI_log10_t2equad',
 'B1855+09_L-wide_ASP_log10_t2equad',
 'B1855+09_L-wide_PUPPI_log10_t2equad']

In [24]:
m.logL(ds.sample_uniform(m.logL.params))

Array(97129.83886491, dtype=float64)

#### Measurement noise only, nanograv backends, parameters from noisedict

In [25]:
psr.noisedict

{'B1855+09_430_ASP_efac': 1.115935306813982,
 'B1855+09_430_ASP_log10_t2equad': -7.564164330699591,
 'B1855+09_430_PUPPI_efac': 1.000049037085653,
 'B1855+09_430_PUPPI_log10_t2equad': -6.572540211467256,
 'B1855+09_L-wide_ASP_efac': 1.043114017270374,
 'B1855+09_L-wide_ASP_log10_t2equad': -6.517929916655293,
 'B1855+09_L-wide_PUPPI_efac': 1.1118432332882,
 'B1855+09_L-wide_PUPPI_log10_t2equad': -7.755603780476984,
 'B1855+09_430_ASP_log10_ecorr': -6.798122106550257,
 'B1855+09_430_PUPPI_log10_ecorr': -5.6989064141929715,
 'B1855+09_L-wide_ASP_log10_ecorr': -6.120457109433745,
 'B1855+09_L-wide_PUPPI_log10_ecorr': -6.641667916624413,
 'B1855+09_red_noise_log10_A': -13.940953818371378,
 'B1855+09_red_noise_gamma': -3.68432133461766}

In [26]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict)])

In [27]:
m.logL.params

[]

In [28]:
m.logL(ds.sample_uniform(m.logL.params))

np.float64(96686.81011373615)

#### Add ECORR noise (GP), free params

In [65]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr),
                         ds.makegp_ecorr(psr)])

In [66]:
m.logL.params

['B1855+09_430_ASP_efac',
 'B1855+09_430_ASP_log10_ecorr',
 'B1855+09_430_ASP_log10_t2equad',
 'B1855+09_430_PUPPI_efac',
 'B1855+09_430_PUPPI_log10_ecorr',
 'B1855+09_430_PUPPI_log10_t2equad',
 'B1855+09_L-wide_ASP_efac',
 'B1855+09_L-wide_ASP_log10_ecorr',
 'B1855+09_L-wide_ASP_log10_t2equad',
 'B1855+09_L-wide_PUPPI_efac',
 'B1855+09_L-wide_PUPPI_log10_ecorr',
 'B1855+09_L-wide_PUPPI_log10_t2equad']

In [67]:
m.logL(ds.sample_uniform(m.logL.params))

Array(95422.18669251, dtype=float64)

#### Add ECORR noise (GP), noisedict params

In [33]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict)])

In [34]:
m.logL.params

[]

In [35]:
m.logL(ds.sample_uniform(m.logL.params))

np.float64(103037.9457516601)

#### Add timing model

In [36]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict),
                         ds.makegp_timing(psr, svd=True)])

In [38]:
m.logL.params

[]

In [39]:
m.logL(ds.sample_uniform(m.logL.params))

np.float64(94817.46786020289)

#### Add red noise (powerlaw)

In [51]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict),
                         ds.makegp_timing(psr, svd=True),
                         ds.makegp_fourier(psr, ds.powerlaw, components=30, name='rednoise')])

In [52]:
m.logL.params

['B1855+09_rednoise_log10_A', 'B1855+09_rednoise_gamma']

In [53]:
m.logL(ds.sample_uniform(m.logL.params))

Array(94817.46858283, dtype=float64)

#### Add red noise (powerlaw, fixed gamma)

In [54]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict),
                         ds.makegp_timing(psr, svd=True),
                         ds.makegp_fourier(psr, ds.partial(ds.powerlaw, gamma=4.33), components=30, name='rednoise')])

In [55]:
m.logL.params

['B1855+09_rednoise_log10_A']

In [56]:
m.logL(ds.sample_uniform(m.logL.params))

Array(94818.95650736, dtype=float64)

#### Add red noise (free spectrum)

In [58]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict),
                         ds.makegp_timing(psr, svd=True),
                         ds.makegp_fourier(psr, ds.freespectrum, components=30, name='rednoise')])

In [59]:
m.logL.params

['B1855+09_rednoise_log10_rho(30)']

In [64]:
m.logL({'B1855+09_rednoise_log10_rho(30)': 1e-6 * np.random.randn(30)})

Array(94024.38714473, dtype=float64)

### Multiple pulsars

In [69]:
psrs = allpsrs[:3]

#### Combined likelihood

In [77]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True),
                                             ds.makegp_fourier(psr, ds.powerlaw, components=30, name='rednoise')])
                        for psr in psrs])

In [78]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A']

In [79]:
m.logL(ds.sample_uniform(m.logL.params))

Array(442105.13787401, dtype=float64)

#### Add common noise

Indicating parameters under common shares them among pulsars

In [ ]:
T = ds.getspan(psrs)

In [115]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True),
                                             ds.makegp_fourier(psr, ds.powerlaw, components=30, T=T, name='rednoise'),
                                             ds.makegp_fourier(psr, ds.powerlaw, components=14, T=T, name='crn',
                                                               common=['crn_log10_A', 'crn_gamma'])])
                        for psr in psrs])

In [116]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'crn_gamma',
 'crn_log10_A']

In [117]:
p0 = ds.sample_uniform(m.logL.params)
m.logL(p0)

Array(487234.62649703, dtype=float64)

#### Parallelize red components

Coordinated timespan is required

In [118]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True)]) for psr in psrs],
                       commongp = [ds.makecommongp_fourier(psrs, ds.powerlaw, components=30, T=T, name='rednoise'),
                                   ds.makecommongp_fourier(psrs, ds.powerlaw, components=14, T=T, name='crn',
                                                           common=['crn_log10_A', 'crn_gamma'])])

In [119]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'crn_gamma',
 'crn_log10_A']

In [120]:
m.logL(p0)

Array(487234.62649703, dtype=float64)

#### Reuse Fourier vectors

`ds.makepowerlaw_crn` yields the sum of two powerlaws, with possibly different number of components.

In [95]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True)]) for psr in psrs],
                       commongp = ds.makecommongp_fourier(psrs, ds.makepowerlaw_crn(components=14), components=30, T=T, name='rednoise',
                                                          common=['crn_log10_A', 'crn_gamma']))

In [96]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'crn_gamma',
 'crn_log10_A']

In [97]:
m.logL(p0)

Array(476084.74919831, dtype=float64)

#### Add global spatially correlated process

Note `ds.makeglobalgp_fourier` requires the ORF, but not the `common` specification, which is automatic.

In [124]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True)]) for psr in psrs],
                       commongp = ds.makecommongp_fourier(psrs, ds.powerlaw, components=30, T=T, name='rednoise'),
                       globalgp = ds.makeglobalgp_fourier(psrs, ds.powerlaw, ds.hd_orf, components=14, T=T, name='gw'))

In [125]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'gw_gamma',
 'gw_log10_A']

In [126]:
p0 = ds.sample_uniform(m.logL.params)
m.logL(p0)

Array(466332.09973769, dtype=float64)

#### Another way of doing this (useful if variable GPs differ among pulsars)

In [127]:
m = ds.GlobalLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True),
                                             ds.makegp_fourier(psr, ds.powerlaw, components=30, name='rednoise')]) for psr in psrs],
                        globalgp = ds.makeglobalgp_fourier(psrs, ds.powerlaw, ds.hd_orf, components=14, T=T, name='gw'))

In [128]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'gw_gamma',
 'gw_log10_A']

In [129]:
m.logL(p0)

Array(466332.19059264, dtype=float64)